In [1]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [2]:
import OpenAI from "openai";

const openai = new OpenAI({
    apiKey: env["ALIBABA_API_KEY"],
    baseURL: `https://dashscope.aliyuncs.com/compatible-mode/v1`,
});

const result = await openai.chat.completions.create({
    model: 'qwen-plus',
    messages: [{ role: 'user', content: 'Say hello!' }],
});

console.log(result.choices[0]!.message?.content);

Hello! How may I assist you today?


In [3]:
function getCurrentWeather({ location, unit="fahrenheit"}){
    const  weather_info = {
      "location": location,
      "temperature": "72",
      "unit": unit,
      "forecast": ["sunny", "windy"],
    }
    return JSON.stringify(weather_info);
}

In [4]:
const tools = [
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "获取指定地点的天气信息",
        parameters: {
          type: "object",
          properties: {
            location: {
              type: "string",
              description: "城市和地点",
            },
            unit: { type: "string", enum: ["celsius", "fahrenheit"] },
          },
          required: ["location"],
        },
      },
    }
]

In [5]:
const messages = [
    {
        "role": "user",
        "content": "上海的天气怎么样"
    }
];

const result = await openai.chat.completions.create({
    model: 'qwen-plus',
    messages,
    tools,
    tool_choice: {
        type: "function",
        function: {
           name: "getCurrentWeather"
        }
    },
});

console.log(result.choices[0]);

Error: 400  'tool_choice' is not support now

In [6]:
function getCurrentTime({ format = "iso" } = {}) {
    let currentTime;
    switch (format) {
      case "iso":
        currentTime = new Date().toISOString();
        break;
      case "locale":
        currentTime = new Date().toLocaleString();
        break;
      default:
        currentTime = new Date().toString();
        break;
    }
    return currentTime;
}

In [7]:
const tools = [
    {
      type: "function",
      function: {
        name: "getCurrentTime",
        description: "Get the current time in a given format",
        parameters: {
          type: "object",
          properties: {
            format: {
              type: "string",
              enum: ["iso", "locale", "string"],
              description: "The format of the time, e.g. iso, locale, string",
            },
          },
          required: ["format"],
        },
      },
    },
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: {
          type: "object",
          properties: {
            location: {
              type: "string",
              description: "The city and state, e.g. San Francisco, CA",
            },
            unit: { type: "string", enum: ["celsius", "fahrenheit"] },
          },
          required: ["location", "unit"],
        },
      },
    },
  ]

In [17]:
const messages = [
    {
        "role": "user",
        "content": "请同时告诉我当前的时间和上海的天气"
    }
]

const result = await openai.chat.completions.create({
    model: 'qwen-plus',
    messages,
    tools,
});

console.log(result.choices[0]);

{
  message: {
    role: "assistant",
    tool_calls: [
      {
        function: { name: "getCurrentTime", arguments: '{"format": "iso"}' },
        id: "",
        type: "function"
      }
    ],
    content: ""
  },
  finish_reason: "tool_calls",
  index: 0,
  logprobs: null
}


In [18]:
const messages = [
    {
      "role": "user",
      "content": "上海的天气怎么样"
    }
];

const result = await openai.chat.completions.create({
    model: 'qwen-plus',
    messages,
    tools,
});

In [19]:
  messages.push(result.choices[0].message)

  const functions = {
    "getCurrentWeather": getCurrentWeather
  }

  const cell = result.choices[0].message.tool_calls[0]
  const functionInfo = cell.function
  const functionName = functionInfo.name;
  const functionParams = functionInfo.arguments
  const functionResult = functions[functionName](functionParams);

  messages.push({
    tool_call_id: cell.id,
    role: "tool",
    name: functionName,
    content: functionResult,
  }); 

3

In [20]:
  const response = await openai.chat.completions.create({
    model: 'qwen-plus',
    messages,
  });

  console.log(response.choices[0].message);

{ role: "assistant", content: ": 上海现在的天气是晴朗并有风，温度约为22℃。" }


In [21]:
import { z } from "zod";

const getCurrentWeatherSchema = z.object({
  location: z.string().describe("The city and state, e.g. San Francisco, CA"), // string 类型
  unit: z.enum(["celsius", "fahrenheit"]).describe("The unit of temperature"), // 枚举类型
});


In [22]:
import { zodToJsonSchema } from "zod-to-json-schema";

const paramSchema = zodToJsonSchema(getCurrentWeatherSchema);

In [27]:
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";

const model = new ChatAlibabaTongyi({
  model: "qwen-plus",
  temperature: 0,
});

// const model = new ChatOpenAI({
//     temperature: 0 
// })

const modelWithTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema),
            }
        }
    ]
})

await modelWithTools.invoke("上海的天气怎么样");
// console.log(res);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "作为一个AI模型，我无法实时获取当前天气信息。建议您查询可靠的天气预报网站或使用手机上的天气应用程序来获取上海最新的天气情况。",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "作为一个AI模型，我无法实时获取当前天气信息。建议您查询可靠的天气预报网站或使用手机上的天气应用程序来获取上海最新的天气情况。",
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 12, completionTokens: 32, totalTokens: 44 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}

In [28]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

  const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant"],
    ["human", "{input}"]
  ])

  const chain = prompt.pipe(modelWithTools)

  const res1 = await chain.invoke({
    input: "上海的天气怎么样",
  });

  console.log(res);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "作为一个AI模型，我无法实时获取当前天气信息。建议您可以通过查看各大天气预报网站，或者使用专业的天气预报应用来查询上海的实时天气情况。",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "作为一个AI模型，我无法实时获取当前天气信息。建议您可以通过查看各大天气预报网站，或者使用专业的天气预报应用来查询上海的实时天气情况。",
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 12, completionTokens: 35, totalTokens: 47 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}


In [29]:
const getCurrentTimeSchema = z.object({
  format: z
    .enum(["iso", "locale", "string"])
    .optional()
    .describe("The format of the time, e.g. iso, locale, string"),
});

zodToJsonSchema(getCurrentTimeSchema);

{
  type: "object",
  properties: {
    format: {
      type: "string",
      enum: [ "iso", "locale", "string" ],
      description: "The format of the time, e.g. iso, locale, string"
    }
  },
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}

In [30]:

const modelWithMultiTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema)
            }
        },
        {
            type: "function",
            function: {
                name: "getCurrentTime",
                description: "Get the current time in a given format",
                parameters: zodToJsonSchema(getCurrentTimeSchema)
            }
        }
    ]
});

In [31]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

  const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant"],
    ["human", "{input}"]
  ])

  const chain = prompt.pipe(modelWithMultiTools)

  const res1 = await chain.invoke({
    input: "上海的天气怎么样",
  });

  console.log(res);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "作为一个AI模型，我无法实时获取当前天气信息。建议您可以通过查看各大天气预报网站，或者使用专业的天气预报应用来查询上海的实时天气情况。",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "作为一个AI模型，我无法实时获取当前天气信息。建议您可以通过查看各大天气预报网站，或者使用专业的天气预报应用来查询上海的实时天气情况。",
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 12, completionTokens: 35, totalTokens: 47 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}


In [32]:
const taggingSchema = z.object({
    emotion:z.enum(["pos", "neg", "neutral"]).describe("文本的情感"),
    language: z.string().describe("文本的核心语言（应为ISO 639-1代码）"),
  });

In [33]:
  const modelTagging = model.bind({
    tools: [
      {
        type: "function",
        function: {
          name: "tagging",
          description: "为特定的文本片段打上标签",
          parameters: zodToJsonSchema(taggingSchema)
        }
      }
    ],
    tool_choice: {
      type: "function",
      function: {
        name: "tagging"
      }
    }
  });

In [34]:
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";

  const prompt = ChatPromptTemplate.fromMessages([
    ["system", "仔细思考，你有充足的时间进行严谨的思考，然后按照指示对文本进行标记"],
    ["human", "{input}"]
  ])

  const chain = prompt.pipe(modelTagging).pipe(new JsonOutputToolsParser());

In [35]:
const res = await chain.invoke({
    input: "hello world"
});
console.log(res);

Error: No tools_call in message [{"message":{"lc":1,"type":"constructor","id":["langchain_core","messages","AIMessage"],"kwargs":{"content":"hello <i>world</i>","tool_calls":[],"invalid_tool_calls":[],"additional_kwargs":{},"response_metadata":{"tokenUsage":{"promptTokens":34,"completionTokens":8,"totalTokens":42}}}},"text":"hello <i>world</i>"}]

In [36]:
  const personExtractionSchema = z.object({
    name: z.string().describe("人的名字"),
    age: z.number().optional().describe("人的年龄")
  }).describe("提取关于一个人的信息");

  const relationExtractSchema = z.object({
    people: z.array(personExtractionSchema).describe("提取所有人"),
    relation: z.string().describe("人之间的关系, 尽量简洁")
  });

  const modelExtract = model.bind({
    tools: [
      {
        type: "function",
        function: {
            name: "relationExtract",
            description: "提取数据中人的信息和人的关系",
            parameters: zodToJsonSchema(relationExtractSchema)
        }
      }
    ],
    tool_choice: {
      type: "function",
      function: {
        name: "relationExtract"
      }
    }
  });

  const prompt = ChatPromptTemplate.fromMessages([
    ["system", "仔细思考，你有充足的时间进行严谨的思考，然后提取文中的相关信息，如果没有明确提供，请不要猜测，可以仅提取部分信息"],
    ["human", "{input}"]
  ]);

  const chain = prompt.pipe(modelExtract).pipe(new JsonOutputToolsParser());

In [37]:
  await chain.invoke({
    input: "小明现在 18 岁了，她妈妈是小丽"
  });

Error: No tools_call in message [{"message":{"lc":1,"type":"constructor","id":["langchain_core","messages","AIMessage"],"kwargs":{"content":"小明妈妈：小丽","tool_calls":[],"invalid_tool_calls":[],"additional_kwargs":{},"response_metadata":{"tokenUsage":{"promptTokens":59,"completionTokens":6,"totalTokens":65}}}},"text":"小明妈妈：小丽"}]